In [59]:
import pandas as pd
import numpy as np
import re
from ast import literal_eval
from datetime import datetime
from pandas import *
pd.set_option('display.float_format', lambda x: '%.2f' % x)
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

In [60]:
# - Create this w/ user input for their user_name, chess_data_file (.pgn as format),
#   & time back to go to analyse - just stats of their data
# - Create a program that asks user for their user_name, chess_data_file, time back to go to predict,
#   move by move gives them a % chance of winning.

# - 'checks' <- count # of checks
# - 'opp_checks' <- count # of checks
# - 'first_check' <- get first column of check
# - 'opp_first_check' <- get first column of check
# - Correct the time!
# - 'score_5' - Score after first 5 moves
# - 'score_10' - Score after first 10 moves
# - 'score_15' - Score after first 15 moves
# - 'score_20' - Score after first 20 moves
# - 'score_25' - Score after first 25 moves
# - 'score_30' - Score after first 30 moves
# - 'score_35' - Score after first 35 moves
# - 'score_40' - Score after first 40 moves
# - 'score_45' - Score after first 45 moves
# - 'score_50' - Score after first 50 moves
# - 'score_55' - Score after first 55 moves
# - 'score_60' - Score after first 60 moves

# ### Future Considered Variables:
# > - y = 'win%' <- Prediction
# - 'timed_game': user_input -- *'enter time in minutes per player (10 for 10 min, 5 for 5 min, etc.): '*
# - 'color': user_input -- *'enter your peices color: '*
# - 'pre_diff': user_input
# - 'Current_move_count':
# - 'delta_avg_move_time' <- Create column from subtracting last time and total time devide by # of moves
# - 'delta_max_move_time' <- Create column from max of all move times
# - 'delta_castled_on_move' <- Create column that looks for '0-0' or '0-0-0'
# - 'score' - has to keep updating after every move
# - 'weighted_score'
#     - 'time': result-0.1 for win (Final_score<=-5), result+0.1 for loss (Final_score>=5)
#     - 'abandoned': result-0.2 for win (Final_score<=-5), result+0.2 for loss (Final_score>=5)
#     - 'stalemate': result-0.25 (Final_score<=-5), result+0.25 (Final_score>=5)

In [63]:
def initial_chess_data(filename):
    with open(filename, 'r+') as file:
        icd_l = file.readlines()

    icd_t = " ".join(num for num in icd_l if len(num) > 4)

    return icd_t
chess_text = initial_chess_data('../data/dest.pgn')

In [65]:
def chess_data_cleanup(chess_text):
    chess_text = chess_text.replace('[', "").replace(']', "")
    chess_text = chess_text.replace('\n', ' ')
    chess_text = chess_text.replace('   ', ' ').replace('  ', ' ')
    chess_text = chess_text.replace('... ', 'b":"').replace('. ', 'w":"')
    chess_text = chess_text.replace('", ', '", "').replace(' {%clk ', '^')
    chess_text = chess_text.replace(' {%clk', '^')
    chess_text = chess_text.replace('}', '",').replace('", ', '", "')
    chess_text = chess_text.replace(' Site "Chess.com" D', ', D')
    chess_text = chess_text.replace('Event ', '}~{"Event":')
    chess_text = chess_text.replace('", Date ', '", "Date": ')
    chess_text = chess_text.replace('" Result ', '", "Result": ')
    chess_text = chess_text.replace('" Round ', '", "Round": ')
    chess_text = chess_text.replace('" White ', '", "White": ')
    chess_text = chess_text.replace('" Black ', '", "Black": ')
    chess_text = chess_text.replace('" WhiteElo ', '", "WhiteElo": ')
    chess_text = chess_text.replace('" TimeControl ', '", "TimeControl": ')
    chess_text = chess_text.replace('" EndTime ', '", "EndTime": ')
    chess_text = chess_text.replace('" BlackElo ', '", "BlackElo": ').replace(
        '" Termination ', '", "Termination": ')
    chess_text = chess_text.replace(
        '"Event":"10|0 Blitz", "Date": "2017.02.16", "Round": "5", "White": "TrueMoeG", "Black": "naggvk", "Result": "0-1", "WhiteElo": "784", "BlackElo": "1210", "TimeControl": "600", "EndTime": "12:27:07 PST", "Termination": "naggvk won - game abandoned" }~{', '')
    chess_text = chess_text.replace(
        '"Event":"Live Chess", "Date": "2017.02.20", "Round": "-", "White": "30mate", "Black": "TrueMoeG", "Result": "0-1", "WhiteElo": "820", "BlackElo": "878", "TimeControl": "600", "EndTime": "11:55:45 PST", "Termination": "TrueMoeG won - game abandoned" }~{', '')
    chess_text = chess_text.replace(
        '"Event":"Live Chess", "Date": "2018.07.14", "Round": "-", "White": "Bran17", "Black": "TrueMoeG", "Result": "0-1", "WhiteElo": "1205", "BlackElo": "961", "TimeControl": "300", "EndTime": "18:02:56 PDT", "Termination": "TrueMoeG won - game abandoned" }~{', '')
    chess_text = chess_text.replace(
        '"Event":"Live Chess", "Date": "2017.04.26", "Round": "-", "White": "nerbenator", "Black": "TrueMoeG", "Result": "0-1", "WhiteElo": "829", "BlackElo": "842", "TimeControl": "600", "EndTime": "16:49:05 PDT", "Termination": "TrueMoeG won - game abandoned" }~{', '')
    chess_text = chess_text.replace(
        '"Event":"10|0 Blitz", "Date": "2017.02.16", "Round": "4", "White": "TrueMoeG", "Black": "Shuzakhan", "Result": "0-1", "WhiteElo": "845", "BlackElo": "1183", "TimeControl": "600", "EndTime": "11:58:21 PST", "Termination": "Shuzakhan won by resignation" }~{', '')
    chess_text = chess_text.replace(' PST', '').replace(' PDT', '')
    #chess_text = chess_text.replace('180+2', '180').replace('300+5', '300')
    chess_text = chess_text.replace('   ', ' ').replace('  ', ' ')
    chess_text = chess_text.replace('" 1w":[', '"}~{"1w":[')
    chess_text = chess_text.replace('" 1w":"', '"}~{"1w":"')
    chess_text = chess_text.replace(', "1/2-1/2 }~{', '}~{')
    chess_text = chess_text.replace(', "1-0 }~{', '}~{')
    chess_text = chess_text.replace(', "0-1 }~{', '}~{')
    chess_text = chess_text.replace(', "1-0 ', '}').replace(', "}', '}')
    cl = ''.join([num for num in chess_text]).split("~")
    df = DataFrame(cl, columns=['a'])
    df = df[df['a'].str.len() > 3]
    return df

cdf = chess_data_cleanup(icd_text)

In [43]:
# adf = cdf[cdf.index % 2 == 0]
# adf.to_csv('moves.csv')

In [66]:
def data_cleaning_1(df):
    c_df = DataFrame(data=list(df['a'].apply(literal_eval)))
    c_df['Date'] = c_df['Date'].fillna(method='ffill')
    c_df['EndTime'] = c_df['EndTime'].fillna(method='ffill')
    c_df['date_time'] = to_datetime(c_df['Date'] + ' ' + c_df['EndTime'])
    c_df['Date'] = to_datetime(c_df['Date'])
    c_df['EndTime'] = to_timedelta(c_df['EndTime'])
    m_df = c_df[c_df.index % 2 == 1]
    m_df = m_df.sort_values('date_time').reset_index().drop(
        columns=['index', 'Date', 'White', 'Black', 'Result', 'WhiteElo',
                 'BlackElo', 'TimeControl', 'EndTime', 'Termination', 'date_time',
                 'Round', 'Event'])

    d_df = c_df[c_df.index % 2 == 0]

    d_df = d_df[['Date', 'White', 'Black', 'Result', 'WhiteElo', 'BlackElo',
                 'TimeControl', 'EndTime', 'Termination', 'date_time'
                 ]].sort_values('date_time').reset_index().drop(columns=[
                     'index', 'date_time'])

    d_df = d_df.rename(
        columns={'Date': 'date', 'White': 'white', 'Black': 'black', 'Result': 'result',
                 'WhiteElo': 'white_elo', 'BlackElo': 'black_elo', 'TimeControl': 'game_time',
                 'EndTime': 'end_time', 'Termination': 'termination'})

    d_df['num_moves'] = m_df.count(axis=1)
    d_df['white_elo'] = to_numeric(d_df['white_elo'])
    d_df['black_elo'] = to_numeric(d_df['black_elo'])
    d_df['color'] = np.where(d_df['white'] == 'TrueMoeG', 1, 0)
    d_df.drop_duplicates(inplace=True)
    m_df.drop_duplicates(inplace=True)
    return m_df, d_df

mdf1, ddf1 = data_cleaning_1(cdf)

In [67]:
def data_cleaning_2(m_df, d_df):
    moves_column_names = ['00' + num if len(num) == 2 else
                          num for num in m_df.columns]
    moves_column_names = ['0' + num if len(num) == 3 else
                          num for num in moves_column_names]
    moves_column_names = [num.replace('w', 'a') for num
                          in moves_column_names]
    m_df.columns = [num for num in moves_column_names]
    moves_column_names = sorted(moves_column_names)
    m_df = m_df[[num for num in moves_column_names]]
    t_df = m_df.copy()
    for col_name in m_df.columns:
        m_df[col_name] = m_df[col_name].str.extract(r'(^\w+-?\w+?-?\w?\+?)')
    for col_name in m_df.columns:
        t_df[col_name] = t_df[col_name].str.extract(r'(\d\:\d+:\d+\.?\d)')
    return m_df, t_df

mdf2, tdf1 = data_cleaning_2(mdf1, ddf1)

In [68]:
def data_cleaning_3(t_df, d_df):
    t_df = t_df.apply(to_timedelta, errors='coerce')
    t_df = t_df.apply(to_numeric, errors='coerce')
    t_df = t_df.div(1000000000)
    t_df['game_time'] = d_df['game_time']
    t_df['extra_time'] = t_df['game_time'].replace([
        '300', '600', '180', '180+2', '300+5'], ['0', '0', '0', '2', '5'])
    t_df['game_time'] = d_df['game_time'].replace(
        ['180+2', '300+5'], [180, 300])
    t_df['extra_time'] = to_numeric(t_df['extra_time'])
    return t_df

tdf2 = data_cleaning_3(tdf1, ddf1)

In [121]:
tdf2.loc[0, '001a']

599.2

In [125]:
def data_cleaning_4(m_df, t_df, d_df):
    t_df['game_time'] = to_numeric(t_df['game_time'])
    d_df['game_time'] = t_df['game_time']
    wh_m_df = m_df[m_df.columns[::2]].copy()
    bl_m_df = m_df[m_df.columns[1::2]].copy()
    wh_t_df = t_df[t_df.columns[::2]].copy()
    bl_t_df = t_df[t_df.columns[1::2]].copy()
    wh_t_df = wh_t_df.drop(columns=[wh_t_df.columns[-1]])
    bl_t_df = bl_t_df.drop(columns=[bl_t_df.columns[-1]])
    d_df['white_num_moves'] = wh_m_df.count(axis=1)
    d_df['black_num_moves'] = bl_m_df.count(axis=1)
    for num in wh_t_df.columns:
        wh_t_df[num] = t_df['game_time'] - wh_t_df[num]
    for num in bl_t_df.columns:
        bl_t_df[num] = t_df['game_time'] - bl_t_df[num]

    two_list = t_df[t_df['extra_time']==2].index.tolist()
    five_list = t_df[t_df['extra_time']==5].index.tolist()

    for num in two_list:
        for i in range(len(wh_t_df.columns)-1):
            wh_t_df.iloc[num, i] = wh_t_df.iloc[num, i] + ((i+1) * 2)
        for j in range(len(wh_t_df.columns)-1):
            bl_t_df.iloc[num, j] = wh_t_df.iloc[num, j] + ((j+1) * 2)

    for num in five_list:
        for i in range(len(wh_t_df.columns)-1):
            wh_t_df.iloc[num, i] = wh_t_df.iloc[num, i] + ((i+1) * 5)
        for j in range(len(wh_t_df.columns)-1):
            bl_t_df.iloc[num, j] = wh_t_df.iloc[num, j] + ((j+1) * 5)

    for num in wh_t_df.columns:
        wh_t_df[num] = np.where(wh_t_df[num] > 5000, 0, wh_t_df[num])
    for num in bl_t_df.columns:
        bl_t_df[num] = np.where(bl_t_df[num] > 5000, 0, bl_t_df[num])

    return wh_m_df, wh_t_df, bl_m_df, bl_t_df, d_df, t_df

wh_mdf1, wh_tdf1, bl_mdf1, bl_tdf1, ddf2, tdf3 = data_cleaning_4(mdf2, tdf2, ddf1)

In [92]:
wh_tdf1.head(60)

,001a,002a,003a,004a,005a,006a,007a,008a,009a,010a,...,119a,120a,121a,122a,123a,124a,125a,126a,127a,128a
0,0.80,4.50,7.90,14.70,28.60,30.30,31.10,34.30,45.80,51.50,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,2.60,9.90,16.70,18.50,23.90,33.30,39.80,44.20,51.90,60.90,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,0.70,3.70,7.00,13.80,16.00,29.00,35.50,66.00,68.10,69.90,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,1.20,2.70,8.90,11.60,26.80,30.20,31.80,82.20,117.50,125.20,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4,11.30,14.80,17.20,20.80,23.30,28.70,36.20,54.70,59.50,119.30,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
5,0.00,1.80,4.00,11.90,13.30,25.10,27.90,30.40,31.70,40.10,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
6,0.00,0.60,10.90,13.50,16.90,19.40,27.20,54.30,60.90,71.40,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
7,1.50,3.20,5.60,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
8,3.10,7.40,12.80,15.00,28.80,32.40,35.30,43.20,57.40,59.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
9,1.10,4.30,16.00,23.10,24.90,33.40,40.20,41.60,46.40,58.50,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [77]:
print(wh_mdf1.info())
print(wh_tdf1.info())
print(bl_mdf1.info())
print(bl_tdf1.info())
print(ddf2.info())
print(tdf3.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2187 entries, 0 to 2217
Columns: 128 entries, 001a to 128a
dtypes: object(128)
memory usage: 2.2+ MB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 2187 entries, 0 to 2217
Columns: 128 entries, 001a to 128a
dtypes: float64(128)
memory usage: 2.2 MB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 2187 entries, 0 to 2217
Columns: 127 entries, 001b to 127b
dtypes: object(127)
memory usage: 2.1+ MB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 2187 entries, 0 to 2217
Columns: 127 entries, 001b to 127b
dtypes: float64(127)
memory usage: 2.1 MB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 2187 entries, 0 to 2217
Data columns (total 13 columns):
date               2187 non-null datetime64[ns]
white              2187 non-null object
black              2187 non-null object
result             2187 non-null object
white_elo          2187 non-null int64
black_elo          2187 non-null int64
game_time          2187

In [124]:
for num in wh_tdf1.columns:
    print(wh_tdf1[num].sum())

1630.1000000000201
7025.9
15071.300000000001
26342.9
39081.2
53257.100000000006
67383.79999999999
83767.79999999999
100227.8
118238.4
136388.8
155665.7
175522.5
196304.0
215324.1
233913.2
252334.8
273448.30000000005
292717.19999999995
309402.6
323547.6
338100.5
350274.69999999995
360776.0
366990.5
375580.4
380787.80000000005
386197.4
388066.5
390145.1
387778.19999999995
380240.69999999995
374851.1
375061.80000000005
369910.9
359800.4
350974.0
341867.3
328441.60000000003
317351.80000000005
309028.9
301938.1
295172.4
284991.19999999995
277772.6
269519.7
254476.99999999997
245612.2
236882.2
226213.0
210912.7
200701.2
190202.7
181102.3
169294.3
160658.60000000003
149562.6
139715.1
132309.0
124155.0
117710.4
108043.3
98212.90000000001
89609.0
84606.0
77287.9
69642.09999999999
64280.100000000006
57474.3
54837.59999999999
47886.8
42809.0
38856.8
36643.1
34657.8
34241.6
31249.4
26456.800000000003
23738.100000000002
20782.300000000003
19399.9
18919.1
18382.0
16449.5
16011.300000000001
14679.9
1

In [48]:


# wh_tmdf1 = data_cleaning_5(wh_tdf1, tdf3, ddf2, '001a')
# bl_tmdf1 = data_cleaning_5(bl_tdf1, tdf3, ddf2, '001b')

In [49]:
# def data_cleaning_5(m_df):
#     s_df_l = m_df.copy()
#     s_df_f = s_df_l.copy()
#     for col_name in s_df_l.columns:
#         s_df_l[col_name] = s_df_l[col_name].str.extract(r'(x\w+\+?$)')
#         s_df_f[col_name] = s_df_f[col_name].str.replace(r'(\w+x)','')
#     return s_df_l

# wh_sdf1 = data_cleaning_5(wh_mdf1)
# list(wh_sdf1.iloc[1587])

In [52]:


# ddf3 = data_cleaning_6(ddf2, mdf2, bl_mdf1, wh_mdf1, wh_tdf1, bl_tdf1)

In [55]:
# analysis_labels = [
#     'date', 'day', 'weekday', 'start_time', 'game_time', 'color', 'elo', 
#     'opp_elo', 'diff', 'result', 'won_by', 'num_moves', 'opp_num_moves', 
#     'avg_time', 'opp_avg_time', 'castled_on', 'opp_castled_on', 'castled',
#     'opp_castled', 'time_used', 'opp_time_used', 'max_move', 'opp_max_move']

# predictions_labels = ['result', 'diff', 'opp_elo', 'elo', 'game_time',
#                       'color', 'start_time', 'day', 'weekday']
# ddf_final_without_scores = data_cleaning_6(ddf3, wh_tmdf1, bl_tmdf1)

In [56]:
# ddf_model = ddf_final_without_scores[predictions_labels].copy()
# ddf_analysis = ddf_final_without_scores[analysis_labels].copy()

In [57]:
# mdf2.to_csv('../data/moves.csv')

In [58]:
# ddf_final_without_scores.to_csv('../data/ddf_final_without_scores.csv')
# ddf_model.to_csv('../data/ddf_model.csv')
# ddf_analysis.to_csv('../data/ddf_analysis.csv')

In [25]:
df_final, df_for_model, df_for_analysis = main_cleanup('../data/dest.pgn')
df_final.to_csv('../data/df_final.csv')
df_for_model.to_csv('../data/df_for_model.csv')
df_for_analysis.to_csv('../data/df_for_analysis.csv')

/home/guess/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [26]:
# Without main function:
# Total_time = (21+13+47+326+78+896+1040+680+3040+105+18+24+12+538+19+120+20+111+170)/1000
# Total_time = 7.278 Seconds

# With main function:
# Total_time = (17+22+15+19+14+19+16+17+22+16+13+17+14+25+20+17+6650)
# Total_time = 6.933 Seconds

# Time_saved = 0.345 Seconds

### Analysis:###

1. 'date'
2. 'result'
3. 'game_time'
4. 'end_time'
5. 'num_moves'
6. 'color'
7. 'won_by':
    - checkmate = 8
    - resignation = 7
    - time = 6
    - material = 5
    - agreement = 4
    - repetition = 3
    - abandoned = 2
    - stalemate = 1
    - rule = 0
8. 'weekday'
    - Monday = 0
    - Tuesday = 1
    - Wednesday = 2
    - Thursday = 3
    - Friday = 4|
    - Saturday = 5
    - Sunday = 6
9. 'day'
10. 'castled_on'
11. 'opp_castled_on'
12. 'castled'
13. 'opp_castled'
14. 'time_used'
15. 'opp_time_used'
16. 'opp_num_moves'
17. 'avg_time'
18. 'opp_avg_time'
19. 'max_time'
20. 'opp_max_time'
21. 'elo'
22. 'opp_elo'
23. 'diff'


__X = 'date'X2, 'weekday', 'day', 'won_by', 'castled_on', 'opp_castled_on', 'time_used', 'opp_time_used', 'opp_num_moves', 'avg_time', 'opp_avg_time', 'max_time', 'opp_max_time', 'elo', 'opp_elo', 'diff'__

__y = 'result', 'color', 'castled', 'opp_castled'__

__Nothing for timecontrol, unless another variable asks__

**All the models have to run avg and cumsum**


### Logistic Model ###
__y = 'result'__
__x__:
1. 'elo'
2. 'opp_elo'
3. 'diff'
4. 'color'
5. 'game_time'
6. 'start_time'
7. 'weekday'
8. 'day'

### Random Tree Forest ###

### Logistic Model ###
__y = 'result'__
__x__:
1. 'elo' (Bin?)
2. 'opp_elo' (Bin?)
3. 'diff' (Bin?)
4. 'color' <- 1 & 0
5. 'game_time' <- Many...??
6. 'start_time'(Bin?)
7. 'weekday' <- 7
8. 'day' <- 31!!

### ?? ###

